# Environment Setup (Kaggle)
This cell clones the repository and sets the working directory.

In [ ]:
import os
# --- Git Clone & Setup ---
REPO_URL = "https://github.com/DanielQH07/tranSTR_Casual.git" # Replace with your repo url if different
REPO_NAME = "tranSTR_Casual"
BRANCH = "main" # or master

if not os.path.exists(REPO_NAME):
    print(f"Cloning {REPO_URL}...")
    !git clone {REPO_URL}
else:
    print("Repo already exists.")

# Change Directory to the repo root for relative imports to work
if os.path.basename(os.getcwd()) != REPO_NAME:
    try:
        # Try searching for causalvid subdirectory first as per common structures
        target_dir = os.path.join(os.getcwd(), REPO_NAME, "causalvid")
        if os.path.exists(target_dir):
             os.chdir(target_dir)
        elif os.path.exists(REPO_NAME):
             os.chdir(REPO_NAME)
        
        print(f"Changed directory to: {os.getcwd()}")
    except Exception as e:
             print(f"Could not set working directory: {e}")

In [ ]:
import os
import torch
import numpy as np
import pandas as pd
import argparse
from torch.utils.data import DataLoader
from utils.util import set_seed, set_gpu_devices, save_file
from networks.model import VideoQAmodel
from DataLoader import VideoQADataset
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from utils.logger import logger

# Import training functions from train.py to ensure consistency
from train import train, eval, predict

print("Libraries imported successfully.")

## 0. Download Data (Kaggle Setup)
We need to download:
1. **Object Features**: `objbox.tar.gz` from Hugging Face.
2. **Annotations & Splits**: Assuming these are in another dataset or need to be downloaded. **PLEASE UPDATE `ANNO_URL` IF NEEDED**.

In [ ]:
# --- Data Configuration ---
HF_OBJ_URL = "https://huggingface.co/datasets/DanielQ07/kltn/resolve/main/objbox.tar.gz"
# TODO: Add URL for Annotations/Splits if they are not in the git repo
# HF_ANNO_URL = "..."

import os
import tarfile
import urllib.request

# Define Output Directories
BASE_WORK_DIR = "/kaggle/working" if os.path.exists("/kaggle/working") else os.getcwd()
OBJ_DIR = os.path.join(BASE_WORK_DIR, "features", "objects")
ANNO_DIR = os.path.join(BASE_WORK_DIR, "data", "annotations") # For sample_list_path

for d in [OBJ_DIR, ANNO_DIR]:
    if not os.path.exists(d): os.makedirs(d)

print(f"Object Features will be in: {OBJ_DIR}")
print(f"Annotations should be in: {ANNO_DIR}")

# 1. Download Object Features
try:
    tar_name = "objbox.tar.gz"
    tar_path = os.path.join(OBJ_DIR, tar_name)
    if HF_OBJ_URL:
        print(f"Downloading {tar_name}...")
        urllib.request.urlretrieve(HF_OBJ_URL, tar_path)
        
        if os.path.exists(tar_path):
            print("Extracting...")
            with tarfile.open(tar_path, "r:gz") as tar:
                tar.extractall(path=OBJ_DIR)
            print("Extraction Complete.")
            # Cleanup
            os.remove(tar_path)
except Exception as e:
    print(f"Error downloading/extracting objects: {e}")

# 2. (Optional) Download Annotations if needed
# If your annotations are in the same repo, you can point to them directly.
# Otherwise, implement similar download logic here.

# Configuration and Path Setup

**Crucial Updates:**
- **start_list_path**: Updated to point to the parent directory containing video_id folders (`text.json`, `answer.json`).
- **object_feature_path**: Points to the extracted object features.

In [ ]:
class Config:
    # --- PATHS ---
    # 1. Video Features (ViT features)
    # Ensure this points to where you have stored/downloaded the .pt files
    video_feature_root = r"D:\KLTN\TranSTR\causalvid\features\vit"  
    if not os.path.exists(video_feature_root) and os.path.exists("/kaggle/input"):
        # Example Kaggle Input path - UPDATE THIS
        video_feature_root = "/kaggle/input/your-dataset-name/features/vit"

    # 2. Object Features (Downloaded above)
    object_feature_path = OBJ_DIR

    # 3. Annotations (sample_list_path)
    # Needs to point to directory containing video_id folders with json files
    # Defaulting to a likely location in the repo or the created ANNO_DIR
    sample_list_path = r"D:\KLTN\TranSTR\causalvid\data\vqa\causal\anno"
    if not os.path.exists(sample_list_path):
        # Fallback to repo location if cloned
        repo_anno = os.path.join(os.getcwd(), "..", "data", "vqa", "causal", "anno")
        if os.path.exists(repo_anno):
            sample_list_path = repo_anno
        else:
            # Use the directory we created earlier (user might need to upload data there)
            sample_list_path = ANNO_DIR

    # 4. Splits (train.txt, valid.txt, test.txt)
    split_dir_txt = r"D:\KLTN\TranSTR\causalvid\data\splits"
    if not os.path.exists(split_dir_txt):
         repo_split = os.path.join(os.getcwd(), "..", "data", "splits")
         if os.path.exists(repo_split):
             split_dir_txt = repo_split
    
    print(f"Configured Paths:\n - Video Feat: {video_feature_root}\n - Object Feat: {object_feature_path}\n - Anno Path: {sample_list_path}\n - Split Dir: {split_dir_txt}")

    # Model Params
    v = "v1"
    bs = 8 
    lr = 1e-4
    epoch = 20
    gpu = 0
    dropout = 0.3
    encoder_dropout = 0.3
    patience = 5
    gamma = 0.1
    decay = 1e-4
    
    # Dataset Params
    dataset = 'causal-vid'
    objs = 10 
    n_query = 5
    
    # Transformer
    d_model = 512
    word_dim = 768
    topK_frame = 20
    topK_obj = 10
    num_encoder_layers = 2
    num_decoder_layers = 2
    nheads = 8
    normalize_before = True
    activation = 'gelu'
    
    # Text Encoder
    text_encoder_lr = 1e-5
    freeze_text_encoder = False
    text_encoder_type = "bert-base-uncased"
    text_pool_mode = 1
    
    # Misc
    hard_eval = False
    pos_ratio = 1.0
    neg_ratio = 1.0
    a = 1.0
    use_amp = True
    num_workers = 2 # Reduced for stability
    frame_feat_dim = 768 
    obj_feat_dim = 2048 + 5 

args = Config()
set_gpu_devices(args.gpu)
set_seed(999)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

## 1. Initialize DataLoaders
The DataLoader has been updated to parse the new JSON folder structure automatically.

In [ ]:
print("Initializing Datasets...")

try:
    # TRAIN
    train_dataset = VideoQADataset(
        split='train', 
        n_query=args.n_query, 
        obj_num=args.objs,
        sample_list_path=args.sample_list_path,
        video_feature_path=args.video_feature_root,
        object_feature_path=args.object_feature_path,
        split_dir=args.split_dir_txt 
    )

    # VAL
    val_dataset = VideoQADataset(
        split='val', 
        n_query=args.n_query, 
        obj_num=args.objs,
        sample_list_path=args.sample_list_path,
        video_feature_path=args.video_feature_root,
        object_feature_path=args.object_feature_path,
        split_dir=args.split_dir_txt
    )

    # TEST
    test_dataset = VideoQADataset(
        split='test', 
        n_query=args.n_query, 
        obj_num=args.objs,
        sample_list_path=args.sample_list_path,
        video_feature_path=args.video_feature_root,
        object_feature_path=args.object_feature_path,
        split_dir=args.split_dir_txt
    )

    train_loader = DataLoader(dataset=train_dataset, batch_size=args.bs, shuffle=True, 
                              num_workers=args.num_workers, pin_memory=True)
    val_loader = DataLoader(dataset=val_dataset, batch_size=args.bs, shuffle=False, 
                            num_workers=args.num_workers, pin_memory=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=args.bs, shuffle=False, 
                             num_workers=args.num_workers, pin_memory=True)

    print("\nDatasets initialized successfully.")
except Exception as e:
    print(f"\nError initializing datasets: {e}")
    import traceback
    traceback.print_exc()

## 2. Verify Data Loading
We verify that a batch can be loaded correctly, ensuring dimensions are as expected.

In [ ]:
try:
    if len(train_loader) > 0:
        for sample in train_loader:
            vid_frame_feat, vid_obj_feat, qns_word, ans_word, ans_id, qns_key = sample
            print("--- Data Verification ---")
            print(f"Frame Features Shape: {vid_frame_feat.shape}")
            print(f"Object Features Shape: {vid_obj_feat.shape}")
            print(f"Question Batch Size: {len(qns_word)}")
            print(f"Sample Question: {qns_word[0]}")
            print("--- Verification Passed ---")
            break
    else:
        print("Warning: Train loader is empty. Check your data paths and splits.")
except Exception as e:
    print(f"Data Verification Failed: {e}")

## 3. Model Setup and Training
We initialize the model with the configuration and start the training loop.

In [ ]:
config_dict = {k: v for k, v in Config.__dict__.items() if not k.startswith('__')}
config_dict['device'] = device
# Ensure compatibility with Model constructor
model = VideoQAmodel(**config_dict)

# Optimizer config
param_dicts = [
    {"params": [p for n, p in model.named_parameters() if "text_encoder" not in n and p.requires_grad]},
    {"params": [p for n, p in model.named_parameters() if "text_encoder" in n and p.requires_grad], "lr": args.text_encoder_lr}
]
optimizer = torch.optim.AdamW(params=param_dicts, lr=args.lr, weight_decay=args.decay)
scheduler = ReduceLROnPlateau(optimizer, 'max', factor=args.gamma, patience=args.patience, verbose=True)
model.to(device)
xe = nn.CrossEntropyLoss().to(device)
scaler = torch.cuda.amp.GradScaler(enabled=args.use_amp)

print("Model and Optimizer created.")

In [ ]:
print("Starting Training Loop...")
best_eval_score = 0.0
best_epoch = 1

# Initialize logger dummy/wrapper if needed or use imported
# logger.debug = print # Simple redirect if logger not configured

if len(train_loader) > 0:
    for epoch in range(1, args.epoch + 1):
        print(f"Epoch {epoch}/{args.epoch}")
        train_loss, train_acc = train(model, optimizer, train_loader, xe, device, use_amp=args.use_amp)
        eval_score = eval(model, val_loader, device)
        scheduler.step(eval_score)
        
        print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f} | Val Acc: {eval_score:.2f}")
        
        if eval_score > best_eval_score:
            best_eval_score = eval_score
            best_epoch = epoch
            save_path = f'./models/best_model_epoch_{epoch}.ckpt'
            if not os.path.exists('./models'): os.makedirs('./models')
            torch.save(model.state_dict(), save_path)
            print(f"  Saved new best model to {save_path}")
        
        # Optional: Run test every epoch
        test_score = eval(model, test_loader, device)
        print(f"  Test Acc: {test_score:.2f}")

    print("Training Complete.")
    print(f"Best Val Acc: {best_eval_score:.2f} at Epoch {best_epoch}")
else:
    print("Cannot start training because train_loader is empty.")

## 4. Final Test Evaluation
Load the best model and evaluate on the test set, saving predictions.

In [ ]:
if os.path.exists(f'./models/best_model_epoch_{best_epoch}.ckpt'):
    model.load_state_dict(torch.load(f'./models/best_model_epoch_{best_epoch}.ckpt'))
    print("Loaded best model.")
    
results, test_acc = predict(model, test_loader, device)
print(f"Final Test Accuracy: {test_acc:.2f}%")

# Save results
if not os.path.exists('./prediction'): os.makedirs('./prediction')
result_path = f'./prediction/results_best_{best_epoch}.json'
save_file(results, result_path)
print(f"Predictions saved to {result_path}")